In [1]:
import os
import pandas as pd
import numpy as np

processed_data_path=os.path.join(os.path.pardir,'data','processed')
write_train_path=os.path.join(processed_data_path,'train.csv')
write_test_path=os.path.join(processed_data_path,'test.csv')

train_df=pd.read_csv(write_train_path,index_col='Id')
test_df=pd.read_csv(write_test_path,index_col='Id')

In [4]:
X=train_df.loc[:,'LotFrontage':].as_matrix().astype('float');
y=train_df['SalePrice'].ravel()

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [7]:
print(X.shape)
print(y.shape)

(1460L, 369L)
(1460L,)


In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)

print np.mean(y_train)
print np.mean(y_test)

180808.89897260274
181370.38356164383


In [13]:
import sklearn
from sklearn.dummy import DummyRegressor

In [19]:
model_dummy=DummyRegressor(strategy='mean')
model_dummy.fit(X_train,y_train)
model_dummy.score(X_test,y_test)

-4.5651866949603104e-05

In [23]:
from sklearn.metrics import r2_score
r2_score(y_test,model_dummy.predict(X_test))

-4.5651866949603104e-05

In [25]:
submission_data_path=os.path.join(os.path.pardir,'data','external')
submission_file_path=os.path.join(submission_data_path,'01_dummy.csv')

def get_submission_file(model,filename):
    test_X=test_df.loc[:,'LotFrontage':].as_matrix().astype('float')    
    predictions=model.predict(test_X)
    df_submission=pd.DataFrame({'Id':test_df.index,'SalePrice':predictions})
    submission_data_path=os.path.join(os.path.pardir,'data','external')
    submission_file_path=os.path.join(submission_data_path,filename)
    df_submission.to_csv(submission_file_path,index=False)

In [26]:
get_submission_file(model_dummy,'03_dummy.csv')

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [27]:
%%HTML
<h2>LINEAR REGRESSION MODEL</h2>

In [30]:
from sklearn.linear_model import LinearRegression
model_lr=LinearRegression()
model_lr.fit(X_train,y_train)
model_lr.score(X_test,y_test)

0.4843063391208693

In [32]:

get_submission_file(model_lr,'01_lr.csv')

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [58]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
# standardization
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.fit_transform(X_test)

In [59]:
model_lr=LinearRegression()

In [72]:
model_lr.fit(X_train_scaled,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [73]:
from sklearn.model_selection import GridSearchCV
paramters={'normalize':[True,False],'fit_intercept':[True,False]}
clf=GridSearchCV(model_lr,paramters,cv=15)

In [82]:
clf.fit(X_train_scaled,y_train)

GridSearchCV(cv=15, error_score='raise',
       estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'normalize': [True, False], 'fit_intercept': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [83]:
clf.best_score_

-3.033129211749749e+24

In [84]:
clf.best_params_

{'fit_intercept': False, 'normalize': True}

In [86]:
#clf.score(X_test_scaled ,y_test)
print(r2_score(clf.predict(X_test_scaled),y_test))
clf.predict(X_test)

0.0


array([2.02209283e+21, 1.11396869e+21, 8.37669438e+20, 1.28199887e+21,
       6.06828888e+20, 1.01145176e+21, 1.12287602e+21, 8.38501968e+20,
       2.91657940e+21, 6.33991207e+20, 1.02560772e+21, 1.29933341e+21,
       1.20751133e+21, 6.59000516e+20, 8.06814517e+20, 1.00407457e+21,
       1.21286418e+21, 7.38267232e+20, 9.24056850e+20, 1.09913202e+21,
       9.05353182e+20, 5.99165721e+20, 1.00736996e+21, 8.58745420e+20,
       1.12912349e+21, 1.06749318e+21, 9.35256300e+20, 7.30372273e+20,
       1.34007926e+21, 7.80200992e+20, 1.39430477e+21, 1.12195716e+21,
       6.63629945e+20, 1.35911183e+21, 1.59921664e+21, 8.48196073e+20,
       1.22837221e+21, 8.99468594e+20, 1.23611020e+21, 1.69193386e+21,
       1.75026067e+21, 1.09037770e+21, 9.86308831e+20, 1.43462669e+21,
       1.77716969e+21, 1.50333438e+21, 9.22786278e+20, 1.03590221e+21,
       1.31352392e+21, 7.17600312e+20, 1.75530879e+21, 7.99553167e+20,
       1.01705562e+21, 5.69511692e+20, 1.28947451e+21, 6.30357489e+20,
      

In [65]:
import pickle
model_file_path=os.path.join(os.path.pardir,'models','lr_model.pkl')
scaler_file_path=os.path.join(os.path.pardir,'models','lr_scaler.pkl')

model_file_pickle=open(model_file_path,'wb')
scaler_file_pickle=open(scaler_file_path,'wb')

pickle.dump(clf,model_file_pickle)
pickle.dump(scaler,scaler_file_pickle)

model_file_pickle.close()
scaler_file_pickle.close()

In [66]:
def get_submission_file_new(model,filename):
    test_X=test_df.loc[:,'LotFrontage':].as_matrix().astype('float')  
    test_X_scaled=scaler.fit_transform(test_X)
    predictions=model.predict(test_X)
    df_submission=pd.DataFrame({'Id':test_df.index,'SalePrice':predictions})
    submission_data_path=os.path.join(os.path.pardir,'data','external')
    submission_file_path=os.path.join(submission_data_path,filename)
    df_submission.to_csv(submission_file_path,index=False)

In [67]:
get_submission_file(clf,'02_lr_scaled.csv')

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [89]:
new_lr_model=LinearRegression(normalize=True)

In [98]:
new_lr_model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [108]:
test_X=test_df.loc[:,'LotFrontage':].as_matrix().astype('float')  
predictions=new_lr_model.predict(test_X)
df_submission=pd.DataFrame({'Id':test_df.index,'SalePrice':predictions})
submission_data_path=os.path.join(os.path.pardir,'data','external')
submission_file_path=os.path.join(submission_data_path,'unscaled.csv')
df_submission.to_csv(submission_file_path,index=False)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [116]:
new_lr_model=LinearRegression(normalize=True)
new_lr_model.fit(X_train_scaled,y_train)


test_X=test_df.loc[:,'LotFrontage':].as_matrix().astype('float')  
test_X_scaled=scaler.fit_transform(test_X)
predictions=new_lr_model.predict(test_X_scaled)
df_submission=pd.DataFrame({'Id':test_df.index,'SalePrice':predictions})
submission_data_path=os.path.join(os.path.pardir,'data','external')
submission_file_path=os.path.join(submission_data_path,'scaled.csv')
df_submission.to_csv(submission_file_path,index=False)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [117]:
train_df.SalePrice.mean()

180921.19589041095

In [112]:
train_df.SalePrice.median()

163000.0

In [118]:
predictions

array([5.84762354e+16, 5.13922071e+16, 6.48920909e+16, ...,
       7.18017069e+16, 1.06084996e+17, 9.95410713e+16])